In [1]:
import tensorflow_hub as hub
import tensorflow as tf
elmo = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]

2023-04-16 00:27:29.776675: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 00:27:30.560038: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-16 00:27:36.495638: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46450 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:01:00.0, compute capability: 8.6
2023-04-16 00:27:36.497133: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46450 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6
2023-04-16 00:27:36.498334: I tensorflow/core/common_runtime/gpu

In [2]:

# Provide input tensor and create embeddings
input_tensor = ["my birthday is the best day of the year"]
embeddings_tensor = elmo(tf.constant(input_tensor))
embeddings_tensor["elmo"]

#, signature="default", as_dict=True)

# #with tf.Session() as sess:
#    # sess.run(tf.global_variables_initializer())
#     #embeddings = sess.run(embeddings_tensor)
# print(embeddings_tensor.shape)
# print(embeddings_tensor)


2023-04-16 00:27:41.039352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-16 00:27:41.301856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-16 00:27:42.781362: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


<tf.Tensor: shape=(1, 9, 1024), dtype=float32, numpy=
array([[[-0.05932376, -0.13270235, -0.12809725, ...,  0.05498327,
          0.5519335 , -0.15849958],
        [ 0.06924757,  0.2892886 ,  0.37162474, ..., -0.2211055 ,
          0.67915773,  1.0837383 ],
        [ 0.02628956,  0.1699    , -0.14044148, ..., -0.5370636 ,
          0.9645198 ,  0.7539367 ],
        ...,
        [-0.09164497,  0.19059905, -0.14441213, ..., -0.84102106,
          0.36322337, -0.4123026 ],
        [ 0.40041718, -0.021569  ,  0.38504004, ...,  0.49860597,
          0.07257305, -0.35831073],
        [ 0.10679698,  0.23584764, -0.43082234, ...,  0.0692614 ,
         -0.1473843 ,  0.13418657]]], dtype=float32)>

In [3]:
import torch
torch.tensor(embeddings_tensor["elmo"].numpy().tolist()).shape

torch.Size([1, 9, 1024])

In [4]:
torch.tensor(embeddings_tensor["elmo"].numpy()).mean(dim=1).shape

torch.Size([1, 1024])

In [5]:
from transformers import (
    PretrainedConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
)

class ElmoConfig(PretrainedConfig):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = 1024
        self.num_hidden_layers = 1
        self.is_encoder_decoder = False
        self.architectures = ["Elmo"]

class TfElmoTokenizer(PreTrainedTokenizer):

    def __call__(
        self, text, return_tensors, truncation, return_offsets_mapping, text_target
    ):
        return text

    @staticmethod
    def from_pretrained(path):
        return TfElmoTokenizer()


class TfElmoModel(PreTrainedModel):
    def __init__(self):
        super().__init__(config=ElmoConfig())
        self.elmo_model = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]

    def forward(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
    ):
        embeddings_tensor = self.elmo_model(tf.constant(input_ids))
        elmo_embeddings = embeddings_tensor["elmo"]
        return torch.tensor(elmo_embeddings.numpy())


    @staticmethod
    def from_pretrained(path):
        return TfElmoModel()

sequences = [
    "my birthday is the best day of the year: yes",
    "my birthday is the best day of the year: no",
]

tokenizer = TfElmoTokenizer.from_pretrained("")
model = TfElmoModel.from_pretrained("")
input = tokenizer(sequences, True, None, None, None)
model(input)

/home/kyle/miniconda3/envs/kyle-elk/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-16 00:27:46.558311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'unused_control_flow_input_36' with dtype int32
	 [[{{node unused_control_flow_input_36}}]]
2023-04-16 00:27:47.099551: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'unused_control_flow_input_36' with dtype int32
	 [[{{node unused_control_flow_input_36}}]]
2023-04-16 00:27:47.33

tensor([[[-0.0593, -0.1325, -0.1278,  ..., -0.0417,  0.5802, -0.1217],
         [ 0.0691,  0.2894,  0.3716,  ..., -0.2550,  0.7404,  1.0348],
         [ 0.0263,  0.1700, -0.1403,  ..., -0.5139,  0.9700,  0.7554],
         ...,
         [ 0.4005, -0.0213,  0.3851,  ...,  0.4600, -0.0890, -0.2617],
         [ 0.3011,  0.3643, -0.3625,  ..., -0.0338, -0.0043,  0.4346],
         [-0.2268,  0.3152, -0.5071,  ..., -0.0724, -0.3216, -0.1728]],

        [[-0.0593, -0.1325, -0.1278,  ..., -0.0324,  0.5929, -0.1303],
         [ 0.0691,  0.2894,  0.3716,  ..., -0.2435,  0.7835,  1.0698],
         [ 0.0263,  0.1700, -0.1403,  ..., -0.5469,  0.9499,  0.7168],
         ...,
         [ 0.4005, -0.0213,  0.3851,  ...,  0.3658, -0.0392, -0.2328],
         [ 0.3011,  0.3643, -0.3625,  ...,  0.1529, -0.1047,  0.2572],
         [ 0.1795, -0.0323, -0.7006,  ..., -0.1090, -0.0246,  0.2996]]])

In [6]:
model(input).shape

torch.Size([2, 10, 1024])

In [7]:
torch.mean(model(input), dim=1).shape

torch.Size([2, 1024])

: 